In [ ]:
from notebook_prelude import *
import experiments

In [ ]:
data = collections.defaultdict(lambda: [])
cmap_cache_files = dataset_helper.get_all_cached_graph_datasets(graph_type=TYPE_CONCEPT_MAP)
for file in helper.log_progress(cmap_cache_files):
    dataset = filename_utils.get_dataset_from_filename(file)
    X, Y = dataset_helper.get_dataset_cached(file)
    X = graph_helper.get_graphs_only(X)
    
    all_labels = set(graph_helper.get_all_node_labels(X))
    data['dataset'] += [dataset] * len(all_labels)
    data['labels'] += [str(x) for x in all_labels]

df = pd.DataFrame(data)
df['num_words'] = df['labels'].str.split().apply(len)
df = df.set_index(['dataset', 'labels'])

In [ ]:
num_datasets = len(df.reset_index().dataset.unique())
fig, axes = plt.subplots(ncols=2, nrows=int(np.ceil(num_datasets / 2)), sharex=False)

for ax, (dataset, df_) in zip(axes.flatten()[:num_datasets], df.groupby('dataset')):
    print(dataset)
    df_.reset_index().set_index('labels').num_words.plot(kind='hist', bins=50, ax=ax, title=dataset)
    ax.grid('off')

fig.tight_layout()

In [ ]:
df.groupby('dataset').num_words.describe()

## Single word distribution

In [ ]:
import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english')) | set([',', 'one', 'two'])

In [ ]:
all_labels_splitted = collections.defaultdict(lambda: [])
for idx, df_ in df.reset_index().iterrows():
    dataset = df_.dataset
    if df_.num_words > 1:
        all_labels_splitted[dataset] += df_['labels'].split()

In [ ]:
data = collections.defaultdict(lambda: [])
for dataset, single_words in all_labels_splitted.items():
    c = collections.Counter(single_words)
    data['dataset'] += [dataset] * len(c.keys())
    keys, vals = zip(*c.items())
    data['label'] += keys
    data['occurrences'] += vals

df_single_word_count = pd.DataFrame(data).sort_values('occurrences', ascending=False)
df_single_word_count_no_stopwords = df_single_word_count[df_single_word_count['label'].apply(lambda x: x not in stopwords)]

In [ ]:
num_datasets = len(df_single_word_count_no_stopwords.dataset.unique())
fig, axes = plt.subplots(ncols=2, nrows=int(np.ceil(num_datasets / 2)))
ax = df_single_word_count_no_stopwords.hist(column='occurrences', bins=120, by='dataset', log=True, ax = axes.flatten()[:num_datasets])
for x in axes.flatten():
    x.set_yscale('log')
    x.grid(True,which="both",ls="-")
fig.tight_layout()
#ax.set_xlabel('word occurrences')
#df_single_word_count[df_single_word_count.index.map(lambda x: x not in stopwords)]

In [ ]:
from preprocessing import preprocessing

df_ = df.reset_index()
df_[(df_.dataset=='ng20') & (df_.num_words > 10)]

df_['label_clean'] = df_['labels'].apply(preprocessing.preprocess)

In [ ]:
df_[(df_.dataset=='ng20') & (df_.num_words > 10)]